In [ ]:
from google.colab import drive 
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
dir = "/content/drive/Shared drives/Unstructured Data Analytics/Project"

In [ ]:
import json
from pandas import json_normalize

with open(f'{dir}/candyhunting.json') as f: 
    d = json.load(f) 

candy_df = json_normalize(d['GraphImages']) 

candy_df = candy_df.drop(['comments_disabled', 'gating_info', 'is_video', 'media_preview', 'shortcode', 'taken_at_timestamp', 'thumbnail_resources', 'thumbnail_src', 'username', 'dimensions.height', 'dimensions.width', 'owner.id', 'video_view_count'], axis=1)
candy_df.head()

,__typename,display_url,id,tags,urls,comments.data,edge_media_preview_like.count,edge_media_to_caption.edges,edge_media_to_comment.count
0,GraphImage,https://scontent-dfw5-2.cdninstagram.com/v/t51...,2445255754024597038,[],[https://scontent-dfw5-2.cdninstagram.com/v/t5...,"[{'created_at': 1605717235, 'id': '17968849507...",526,[{'node': {'text': '7-Eleven has new Pomegrana...,11
1,GraphSidecar,https://scontent-dfw5-1.cdninstagram.com/v/t51...,2444788088506347330,[],[https://scontent-dfw5-1.cdninstagram.com/v/t5...,"[{'created_at': 1605661487, 'id': '17859601469...",1605,[{'node': {'text': 'Aldi has Winter Sampler an...,21
2,GraphImage,https://scontent-dfw5-2.cdninstagram.com/v/t51...,2444738912128969833,[],[https://scontent-dfw5-2.cdninstagram.com/v/t5...,"[{'created_at': 1605655625, 'id': '18122663281...",1205,[{'node': {'text': 'Kroger has some new chips ...,32
3,GraphImage,https://scontent-dfw5-2.cdninstagram.com/v/t51...,2444572163421210189,[],[https://scontent-dfw5-2.cdninstagram.com/v/t5...,"[{'created_at': 1605635743, 'id': '17931024274...",2605,[{'node': {'text': 'Gingerbread Man and Snowfl...,46
4,GraphImage,https://scontent-dfw5-1.cdninstagram.com/v/t51...,2444474423974432855,[PepsiApplePieChallenge],[https://scontent-dfw5-1.cdninstagram.com/v/t5...,"[{'created_at': 1605669323, 'id': '18043997473...",5960,[{'node': {'text': 'Pepsi is releasing new Pep...,332


In [ ]:
i=0
count=0
comments = []

while i < len(candy_df):
  while j < len(candy_df['comments.data'][i]):
    comments.append(str(candy_df['comments.data'][i][j]['text']))
    #print(candy_df['comments.data'][i][j]['text'])
    j += 1
  j=0
  i += 1

101512


In [ ]:
from string import punctuation
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk.tokenize import WordPunctTokenizer
import re
#nltk.download('stopwords')
#nltk.download('punkt')


stop = stopwords.words('english')
stop += ['like','dont','would','rt', 'im', 'today', 'http', 'amp', 'https', 'let']

def count(elements, a_dict): 

    # if there exists a key as "elements" then 
    # don't increase its value. Lazy but oh well
    if elements in a_dict: 
        a_dict[elements] += 1
   
    # if the dictionary does not have the key as "elements"  
    # then create a key "elements" and assign its value to 1. 
    else:
        # don't add stopwords or single character elements
        if elements not in stop and len(elements) > 1:
            a_dict.update({elements: 1})

def count2(elements, pair_dict):

    if elements in pair_dict:
        pair_dict[elements] += 1
        
    else:
        pair_dict.update({elements: 1})

In [ ]:
word_freq = {}
final_list = []
pair_freq = {}

for post in comments:
    # here we split the lines into words and append those words to a new list at the end
    post = post.split(' ')
    word_hold = []
    for word in post:
        word = re.sub(r'[^A-Za-z0-9 ]+', '', word)
        word = word.lower()
        word_hold.append(word)

    word_hold = set(word_hold)
    word_hold = list(word_hold)
    
    pairs = []
    for word in word_hold:
        for word2 in word_hold:
            word2 = re.sub(r'[^A-Za-z0-9 ]+', '', word2)
            word2 = word2.lower()
            if word in stop or word2 in stop: continue
            if word == '' or word2 == '': continue
            if word == word2: continue
            pairs.append(tuple(sorted([word, word2])))
        pairs = list(set(pairs))

    for item in pairs:
        count2(item, pair_freq)
    
    for item in word_hold:
        final_list.append(item)

          
for element in final_list:
    count(element, word_freq)

sort_dict = sorted(word_freq.items(), key=lambda x: x[1], reverse=True)
sort_dict_10 = sort_dict[:200]
for i in sort_dict_10:
    print(i[0], i[1])

need 3798
good 3466
omg 3046
try 2716
one 2407
lol 2374
get 2008
oh 1962
want 1728
love 1683
find 1642
yes 1637
back 1395
look 1278
know 1259
candyhunting 1251
chocolate 1221
cant 1199
flavor 1165
looks 1162
think 1019
taste 1018
food 1010
candy 985
oreo 974
go 966
ill 942
got 937
yum 930
ones 920
yummy 906
snack 894
really 879
thats 859
news 855
make 822
delish 818
dessert 790
sounds 790
sweets 763
ive 763
wait 762
still 759
bring 755
macros 748
foodie 748
cream 748
never 747
iifym 746
foodgram 745
junkfood 745
forkyeah 745
instafood 745
flexibledieting 745
foodphotography 745
eeeeeats 745
fitspo 745
foodporn 745
foodstagram 745
see 745
time 740
cheatday 737
foodgasm 736
cheatmeal 736
wow 702
please 698
eat 693
amazing 691
best 681
walmart 677
going 673
better 670
year 662
cookie 651
found 632
thought 627
tried 623
us 621
right 619
sure 614
much 612
favorite 607
new 604
buy 602
even 598
gonna 594
come 578
didnt 574
theyre 574
cereal 573
saw 567
cookies 563
sugar 562
flavors 560
wish 5

In [ ]:
sort_pair = sorted(pair_freq.items(), key=lambda x: x[1], reverse=True)
sort_pair_10 = sort_pair[:10]
for i in sort_pair_10:
    print(i[0], i[1])

('food', 'news') 751
('food', 'snack') 748
('news', 'snack') 746
('foodie', 'snack') 746
('foodie', 'news') 745
('forkyeah', 'iifym') 745
('eeeeeats', 'instafood') 745
('forkyeah', 'macros') 745
('foodporn', 'instafood') 745
('flexibledieting', 'iifym') 745


In [ ]:
sort_pair[0][0][0]
separated_pairs = []

for item in sort_pair:
  word1 = item[0][0]
  word2 = item[0][1]
  freq = item[1]
  separated_pairs.append([word1, word2, freq])

separated_pairs[:10]

[['food', 'news', 751],
 ['food', 'snack', 748],
 ['news', 'snack', 746],
 ['foodie', 'snack', 746],
 ['foodie', 'news', 745],
 ['forkyeah', 'iifym', 745],
 ['eeeeeats', 'instafood', 745],
 ['forkyeah', 'macros', 745],
 ['foodporn', 'instafood', 745],
 ['flexibledieting', 'iifym', 745]]

In [ ]:
import csv

writer = csv.writer(open(f'{dir}/word_freq.csv', 'w'))
for item in sort_dict:
    writer.writerow([item[0], item[1]])

writer2 = csv.writer(open(f'{dir}/word_pair_freq.csv', 'w'))
for item in sort_pair:
    writer2.writerow([item[0], item[1]])

writer3 = csv.writer(open(f'{dir}/sep_word_pair_freq.csv', 'w'))
for item in separated_pairs:
    writer3.writerow([item[0], item[1], item[2]])

## Two Types of Food: Candy and Snack
## Sentiment Words Chosen: Flavor, Chocolate, Taste, Buy, Interesting, Sweet(s), Cream, Pumpkin, Coffee, Cinnamon, Caramel, Strawberry

In [ ]:
word_freq = pd.read_csv("/content/drive/Shared drives/Unstructured Data Analytics/Project/word_freq.csv", index_col=0, header=None, names=['count'])
word_freq.head()

,count
need,3798
good,3466
omg,3046
try,2716
one,2407


In [ ]:
pair_freq = pd.read_csv("/content/drive/Shared drives/Unstructured Data Analytics/Project/sep_word_pair_freq.csv", header=None, names=['word1', 'word2', 'count'])
pair_freq.head()

,word1,word2,count
0,food,news,751
1,food,snack,748
2,news,snack,746
3,foodie,snack,746
4,foodie,news,745


In [ ]:
n = len(candy_df)

def calc_lift(word1, word2, n, counts, pair_counts):
  if word1 == word2:
    return None
  word1, word2 = sorted([word1, word2])
  if not ((pair_freq['word1'] == word1) & (pair_freq['word2'] == word2)).any():
    return 0
  nom = n * pair_counts[(pair_freq['word1'] == word1) & (pair_freq['word2'] == word2)]['count']
  den = counts.loc[word1]['count'] * counts.loc[word2]['count']
  return float(nom / den)

In [ ]:
top_words = ['flavor', 'chocolate', 'taste', 'yummy', 'delish', 'sweets', 'cream', 'pumpkin', 'coffee', 'cinnamon', 'caramel', 'strawberry']
foods = ['candy', 'snack']

matrix_attrs = []
for issue in top_words:
  lifts = []
  for food in foods:
    lifts.append(calc_lift(issue, food, n, word_freq, pair_freq))
  matrix_attrs.append(lifts)

In [ ]:
lift_matrix = pd.DataFrame(matrix_attrs, columns=foods, index=top_words)
lift_matrix

,candy,snack
flavor,0.033333,0.000720
chocolate,0.127839,0.173144
taste,0.030666,0.002472
yummy,0.081521,0.309273
delish,0.249463,0.764059
sweets,0.267445,0.818035
cream,0.022395,0.003365
pumpkin,0.009679,0.074650
coffee,0.012660,0.123547
cinnamon,0.011566,0.029734


In [ ]:
 # Use Vadar for Sentiment Analysis
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyser = SentimentIntensityAnalyzer()

def sentiment_analyzer_scores(sentence):
    score = analyser.polarity_scores(sentence)
    return score

In [ ]:
# TODO: only grab tweets that contain candidate and issue - remove stop words - look at window around candidate and issue 
# sentiment_tweets = []
# for post in tweets_filtered:
#     if ('biden' in post or 'trump' in post) and any(issue in post for issue in top_4_issues):
#         sentiment_tweets.append(' '.join(post))
# sentiment_tweets[:5]

candy = dict()
snack = dict()

for food in foods:
    for word in top_words:
        for comment in comments:
            if food in comment and word in comment:
                food_idx = comment.index(food)
                word_idx = comment.index(word)
                i, j = min(food_idx, word_idx), max(food_idx, word_idx)
                if i >= 2:
                    i -= 2
                if j <= len(comment) - 3:
                    j += 2
                span = comment[i:j+1]
                if food == 'candy':
                    if word in candy.keys():
                        candy[word].append(' '.join(span))
                    else:
                        candy[word] = [' '.join(span)]
                if food == 'snack':
                    if word in snack.keys():
                        snack[word].append(' '.join(span))
                    else:
                        snack[word] = [' '.join(span)]

In [ ]:
candy_sents = dict()
snack_sents = dict()

for food in foods:
    for word in top_words:
        if food == 'candy':
            candy_sents[word] = 0
            for post in candy[word]:
                candy_sents[word] += sentiment_analyzer_scores(post)['compound']
            candy_sents[word] /= len(candy[word])
        if food == 'snack':
            snack_sents[word] = 0
            for post in snack[word]:
                snack_sents[word] += sentiment_analyzer_scores(post)['compound']
            snack_sents[word] /= len(snack[word])
print(candy_sents)
print(snack_sents)

{'flavor': 0.6841880281690135, 'chocolate': 0.308770416666666, 'taste': 0.4891020833333332, 'yummy': 0.39589897959183673, 'delish': 0.9186535055350575, 'sweets': 0.9157207407407428, 'cream': 0.35079444444444435, 'pumpkin': 0.41795625000000003, 'coffee': 0.46102857142857145, 'cinnamon': 0.3956222222222222, 'caramel': 0.4391972972972974, 'strawberry': 0.3310444444444444}
{'flavor': 0.6539733333333333, 'chocolate': 0.8931084942084959, 'taste': 0.4125888888888889, 'yummy': 0.9330561194029868, 'delish': 0.45879999999999693, 'sweets': 0.0, 'cream': 0.8848103773584901, 'pumpkin': 0.9174083333333329, 'coffee': 0.8786215384615388, 'cinnamon': 0.9146826086956519, 'caramel': 0.90304375, 'strawberry': 0.9243777777777777}


In [ ]:
words = candy_sents.keys()
candy_vals = candy_sents.values()
snack_vals = snack_sents.values()
data = [list(candy_vals), list(snack_vals)]

In [ ]:
sentiments = pd.DataFrame(data, columns=words, index=['candy', 'snack'])
sentiments

,flavor,chocolate,taste,yummy,delish,sweets,cream,pumpkin,coffee,cinnamon,caramel,strawberry
candy,0.684188,0.308770,0.489102,0.395899,0.918654,0.915721,0.350794,0.417956,0.461029,0.395622,0.439197,0.331044
snack,0.653973,0.893108,0.412589,0.933056,0.458800,0.000000,0.884810,0.917408,0.878622,0.914683,0.903044,0.924378


In [ ]:
def get_labels(row):
    path = row[1]
    image = vision.Image()
    image.source.image_uri = path
    response = client.label_detection(image=image)
    labels = response.label_annotations
    labels = [label.description for label in labels]
    return labels

In [ ]:
import io
import os

os.environ["GOOGLE_APPLICATION_CREDENTIALS"]=f'{dir}/vision-key.json'

# Imports the Google Cloud client library
from google.cloud import vision

# Instantiates a client
client = vision.ImageAnnotatorClient()

candy_df['labels'] = candy_df.apply(get_labels, axis=1)
candy_df.head()

In [ ]:
candy_df.to_csv(f'{dir}/candy_labeled.csv')

In [ ]:
labels_df = pd.read_csv(f'{dir}/candy_labeled.csv')
labels_df

,Unnamed: 0,__typename,display_url,id,tags,urls,comments.data,edge_media_preview_like.count,edge_media_to_caption.edges,edge_media_to_comment.count,labels
0,0,GraphImage,https://scontent-dfw5-2.cdninstagram.com/v/t51...,2445255754024597038,[],['https://scontent-dfw5-2.cdninstagram.com/v/t...,"[{'created_at': 1605717235, 'id': '17968849507...",526,[{'node': {'text': '7-Eleven has new Pomegrana...,11,['Brush']
1,1,GraphSidecar,https://scontent-dfw5-1.cdninstagram.com/v/t51...,2444788088506347330,[],['https://scontent-dfw5-1.cdninstagram.com/v/t...,"[{'created_at': 1605661487, 'id': '17859601469...",1605,[{'node': {'text': 'Aldi has Winter Sampler an...,21,"['Food', 'Material property', 'Junk food', 'Fo..."
2,2,GraphImage,https://scontent-dfw5-2.cdninstagram.com/v/t51...,2444738912128969833,[],['https://scontent-dfw5-2.cdninstagram.com/v/t...,"[{'created_at': 1605655625, 'id': '18122663281...",1205,[{'node': {'text': 'Kroger has some new chips ...,32,"['Food', 'Junk food', 'Snack', 'Potato chip', ..."
3,3,GraphImage,https://scontent-dfw5-2.cdninstagram.com/v/t51...,2444572163421210189,[],['https://scontent-dfw5-2.cdninstagram.com/v/t...,"[{'created_at': 1605635743, 'id': '17931024274...",2605,[{'node': {'text': 'Gingerbread Man and Snowfl...,46,"['Product', 'Plastic']"
4,4,GraphImage,https://scontent-dfw5-1.cdninstagram.com/v/t51...,2444474423974432855,['PepsiApplePieChallenge'],['https://scontent-dfw5-1.cdninstagram.com/v/t...,"[{'created_at': 1605669323, 'id': '18043997473...",5960,[{'node': {'text': 'Pepsi is releasing new Pep...,332,"['Bottle', 'Product', 'Drink', 'Liquid', 'Choc..."
...,...,...,...,...,...,...,...,...,...,...,...
745,745,GraphImage,https://scontent-dfw5-1.cdninstagram.com/v/t51...,2240405160927406173,[],['https://scontent-dfw5-1.cdninstagram.com/v/t...,"[{'created_at': 1581371175, 'id': '17867363203...",8347,"[{'node': {'text': ""Here are the new Froot Loo...",182,"['Snack', 'Food', 'Junk food', 'Vegetarian foo..."
746,746,GraphImage,https://scontent-dfw5-2.cdninstagram.com/v/t51...,2240258406374389117,[],['https://scontent-dfw5-2.cdninstagram.com/v/t...,"[{'created_at': 1581408741, 'id': '17861260801...",8393,"[{'node': {'text': ""Orange Vanilla Creme M&M's...",202,"['Food', 'Snack']"
747,747,GraphImage,https://scontent-dfw5-1.cdninstagram.com/v/t51...,2240174777933537335,[],['https://scontent-dfw5-1.cdninstagram.com/v/t...,"[{'created_at': 1581269675, 'id': '17891124265...",4388,"[{'node': {'text': ""New The Incredibles Jack-J...",44,"['Frozen dessert', 'Food', 'Ice cream', 'Choco..."
748,748,GraphSidecar,https://scontent-dfw5-1.cdninstagram.com/v/t51...,2239603312410741737,[],['https://scontent-dfw5-1.cdninstagram.com/v/t...,"[{'created_at': 1581211627, 'id': '17946710461...",3689,[{'node': {'text': 'New Crush Fruit Mix flavor...,69,"['Junk food', 'Snack', 'Food', 'Potato chip', ..."


In [ ]:
labels_df.drop(['urls'], axis=1, inplace=True)
labels_df = labels_df.rename(columns={'__typename': 'type','display_url': 'URL', 'comments.data': 'comments', 'edge_media_to_caption.edges':'caption', 'edge_media_preview_like.count':'likes_count', 'edge_media_to_comment.count':'comments_count'})

labels_df

,type,URL,tags,comments,likes_count,caption,comments_count,labels
0,GraphImage,https://scontent-dfw5-2.cdninstagram.com/v/t51...,[],"[{'created_at': 1605717235, 'id': '17968849507...",526,[{'node': {'text': '7-Eleven has new Pomegrana...,11,['Brush']
1,GraphSidecar,https://scontent-dfw5-1.cdninstagram.com/v/t51...,[],"[{'created_at': 1605661487, 'id': '17859601469...",1605,[{'node': {'text': 'Aldi has Winter Sampler an...,21,"['Food', 'Material property', 'Junk food', 'Fo..."
2,GraphImage,https://scontent-dfw5-2.cdninstagram.com/v/t51...,[],"[{'created_at': 1605655625, 'id': '18122663281...",1205,[{'node': {'text': 'Kroger has some new chips ...,32,"['Food', 'Junk food', 'Snack', 'Potato chip', ..."
3,GraphImage,https://scontent-dfw5-2.cdninstagram.com/v/t51...,[],"[{'created_at': 1605635743, 'id': '17931024274...",2605,[{'node': {'text': 'Gingerbread Man and Snowfl...,46,"['Product', 'Plastic']"
4,GraphImage,https://scontent-dfw5-1.cdninstagram.com/v/t51...,['PepsiApplePieChallenge'],"[{'created_at': 1605669323, 'id': '18043997473...",5960,[{'node': {'text': 'Pepsi is releasing new Pep...,332,"['Bottle', 'Product', 'Drink', 'Liquid', 'Choc..."
...,...,...,...,...,...,...,...,...
745,GraphImage,https://scontent-dfw5-1.cdninstagram.com/v/t51...,[],"[{'created_at': 1581371175, 'id': '17867363203...",8347,"[{'node': {'text': ""Here are the new Froot Loo...",182,"['Snack', 'Food', 'Junk food', 'Vegetarian foo..."
746,GraphImage,https://scontent-dfw5-2.cdninstagram.com/v/t51...,[],"[{'created_at': 1581408741, 'id': '17861260801...",8393,"[{'node': {'text': ""Orange Vanilla Creme M&M's...",202,"['Food', 'Snack']"
747,GraphImage,https://scontent-dfw5-1.cdninstagram.com/v/t51...,[],"[{'created_at': 1581269675, 'id': '17891124265...",4388,"[{'node': {'text': ""New The Incredibles Jack-J...",44,"['Frozen dessert', 'Food', 'Ice cream', 'Choco..."
748,GraphSidecar,https://scontent-dfw5-1.cdninstagram.com/v/t51...,[],"[{'created_at': 1581211627, 'id': '17946710461...",3689,[{'node': {'text': 'New Crush Fruit Mix flavor...,69,"['Junk food', 'Snack', 'Food', 'Potato chip', ..."


In [ ]:
labels_df['normalized_likes'] = labels_df['likes_count'] / labels_df['likes_count'].max()
labels_df['normalized_comments'] = labels_df['comments_count'] / labels_df['comments_count'].max()
labels_df.head()

,type,URL,tags,comments,likes_count,caption,comments_count,labels,normalized_likes,normalized_comments
0,GraphImage,https://scontent-dfw5-2.cdninstagram.com/v/t51...,[],"[{'created_at': 1605717235, 'id': '17968849507...",526,[{'node': {'text': '7-Eleven has new Pomegrana...,11,['Brush'],0.014839,0.010816
1,GraphSidecar,https://scontent-dfw5-1.cdninstagram.com/v/t51...,[],"[{'created_at': 1605661487, 'id': '17859601469...",1605,[{'node': {'text': 'Aldi has Winter Sampler an...,21,"['Food', 'Material property', 'Junk food', 'Fo...",0.045279,0.020649
2,GraphImage,https://scontent-dfw5-2.cdninstagram.com/v/t51...,[],"[{'created_at': 1605655625, 'id': '18122663281...",1205,[{'node': {'text': 'Kroger has some new chips ...,32,"['Food', 'Junk food', 'Snack', 'Potato chip', ...",0.033994,0.031465
3,GraphImage,https://scontent-dfw5-2.cdninstagram.com/v/t51...,[],"[{'created_at': 1605635743, 'id': '17931024274...",2605,[{'node': {'text': 'Gingerbread Man and Snowfl...,46,"['Product', 'Plastic']",0.073490,0.045231
4,GraphImage,https://scontent-dfw5-1.cdninstagram.com/v/t51...,['PepsiApplePieChallenge'],"[{'created_at': 1605669323, 'id': '18043997473...",5960,[{'node': {'text': 'Pepsi is releasing new Pep...,332,"['Bottle', 'Product', 'Drink', 'Liquid', 'Choc...",0.168138,0.326450


In [ ]:
labels_df['engagement'] = (labels_df['normalized_likes'] * .4) + (labels_df['normalized_comments'] * .6)
labels_df.head()

,type,URL,tags,comments,likes_count,caption,comments_count,labels,normalized_likes,normalized_comments,engagement
0,GraphImage,https://scontent-dfw5-2.cdninstagram.com/v/t51...,[],"[{'created_at': 1605717235, 'id': '17968849507...",526,[{'node': {'text': '7-Eleven has new Pomegrana...,11,['Brush'],0.014839,0.010816,0.012425
1,GraphSidecar,https://scontent-dfw5-1.cdninstagram.com/v/t51...,[],"[{'created_at': 1605661487, 'id': '17859601469...",1605,[{'node': {'text': 'Aldi has Winter Sampler an...,21,"['Food', 'Material property', 'Junk food', 'Fo...",0.045279,0.020649,0.030501
2,GraphImage,https://scontent-dfw5-2.cdninstagram.com/v/t51...,[],"[{'created_at': 1605655625, 'id': '18122663281...",1205,[{'node': {'text': 'Kroger has some new chips ...,32,"['Food', 'Junk food', 'Snack', 'Potato chip', ...",0.033994,0.031465,0.032477
3,GraphImage,https://scontent-dfw5-2.cdninstagram.com/v/t51...,[],"[{'created_at': 1605635743, 'id': '17931024274...",2605,[{'node': {'text': 'Gingerbread Man and Snowfl...,46,"['Product', 'Plastic']",0.073490,0.045231,0.056535
4,GraphImage,https://scontent-dfw5-1.cdninstagram.com/v/t51...,['PepsiApplePieChallenge'],"[{'created_at': 1605669323, 'id': '18043997473...",5960,[{'node': {'text': 'Pepsi is releasing new Pep...,332,"['Bottle', 'Product', 'Drink', 'Liquid', 'Choc...",0.168138,0.326450,0.263126


In [ ]:
labels_df.drop(['normalized_likes', 'normalized_comments'], axis=1, inplace=True)
labels_df.head()

,type,URL,tags,comments,likes_count,caption,comments_count,labels,engagement
0,GraphImage,https://scontent-dfw5-2.cdninstagram.com/v/t51...,[],"[{'created_at': 1605717235, 'id': '17968849507...",526,[{'node': {'text': '7-Eleven has new Pomegrana...,11,['Brush'],0.012425
1,GraphSidecar,https://scontent-dfw5-1.cdninstagram.com/v/t51...,[],"[{'created_at': 1605661487, 'id': '17859601469...",1605,[{'node': {'text': 'Aldi has Winter Sampler an...,21,"['Food', 'Material property', 'Junk food', 'Fo...",0.030501
2,GraphImage,https://scontent-dfw5-2.cdninstagram.com/v/t51...,[],"[{'created_at': 1605655625, 'id': '18122663281...",1205,[{'node': {'text': 'Kroger has some new chips ...,32,"['Food', 'Junk food', 'Snack', 'Potato chip', ...",0.032477
3,GraphImage,https://scontent-dfw5-2.cdninstagram.com/v/t51...,[],"[{'created_at': 1605635743, 'id': '17931024274...",2605,[{'node': {'text': 'Gingerbread Man and Snowfl...,46,"['Product', 'Plastic']",0.056535
4,GraphImage,https://scontent-dfw5-1.cdninstagram.com/v/t51...,['PepsiApplePieChallenge'],"[{'created_at': 1605669323, 'id': '18043997473...",5960,[{'node': {'text': 'Pepsi is releasing new Pep...,332,"['Bottle', 'Product', 'Drink', 'Liquid', 'Choc...",0.263126


In [ ]:
med = labels_df['engagement'].median()
labels_df['engagement_level'] = labels_df['engagement'].apply(lambda e: 1 if e >= med else 0)
labels_df.head()

,type,URL,tags,comments,likes_count,caption,comments_count,labels,engagement,engagement_level
0,GraphImage,https://scontent-dfw5-2.cdninstagram.com/v/t51...,[],"[{'created_at': 1605717235, 'id': '17968849507...",526,[{'node': {'text': '7-Eleven has new Pomegrana...,11,['Brush'],0.012425,0
1,GraphSidecar,https://scontent-dfw5-1.cdninstagram.com/v/t51...,[],"[{'created_at': 1605661487, 'id': '17859601469...",1605,[{'node': {'text': 'Aldi has Winter Sampler an...,21,"['Food', 'Material property', 'Junk food', 'Fo...",0.030501,0
2,GraphImage,https://scontent-dfw5-2.cdninstagram.com/v/t51...,[],"[{'created_at': 1605655625, 'id': '18122663281...",1205,[{'node': {'text': 'Kroger has some new chips ...,32,"['Food', 'Junk food', 'Snack', 'Potato chip', ...",0.032477,0
3,GraphImage,https://scontent-dfw5-2.cdninstagram.com/v/t51...,[],"[{'created_at': 1605635743, 'id': '17931024274...",2605,[{'node': {'text': 'Gingerbread Man and Snowfl...,46,"['Product', 'Plastic']",0.056535,0
4,GraphImage,https://scontent-dfw5-1.cdninstagram.com/v/t51...,['PepsiApplePieChallenge'],"[{'created_at': 1605669323, 'id': '18043997473...",5960,[{'node': {'text': 'Pepsi is releasing new Pep...,332,"['Bottle', 'Product', 'Drink', 'Liquid', 'Choc...",0.263126,1


In [ ]:
labels_df.to_csv(f'{dir}/candyhunting_engagement.csv')

### Topic Modeling

In [ ]:
df_lda = labels_df.copy()
df_lda.drop(['type', 'URL', 'tags', 'comments', 'caption', 'likes_count', 'comments_count'], axis=1, inplace=True)
df_lda

,labels,engagement,engagement_level
0,['Brush'],0.012425,0
1,"['Food', 'Material property', 'Junk food', 'Fo...",0.030501,0
2,"['Food', 'Junk food', 'Snack', 'Potato chip', ...",0.032477,0
3,"['Product', 'Plastic']",0.056535,0
4,"['Bottle', 'Product', 'Drink', 'Liquid', 'Choc...",0.263126,1
...,...,...,...
745,"['Snack', 'Food', 'Junk food', 'Vegetarian foo...",0.201566,1
746,"['Food', 'Snack']",0.213884,1
747,"['Frozen dessert', 'Food', 'Ice cream', 'Choco...",0.075475,0
748,"['Junk food', 'Snack', 'Food', 'Potato chip', ...",0.082336,0


In [ ]:
from sklearn.decomposition import LatentDirichletAllocation as LDA
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np

count_vectorizer = CountVectorizer(stop_words='english')

# Fit and transform the processed titles
count_data = count_vectorizer.fit_transform(df_lda['labels'])

 
def print_topics(model, count_vectorizer, n_top_words):
    words = count_vectorizer.get_feature_names()
    for topic_idx, topic in enumerate(model.components_):
        print("\nTopic #%d:" % topic_idx)
        print(" ".join([words[i]
                        for i in topic.argsort()[:-n_top_words - 1:-1]]))
        
# Tweak the two parameters below
number_topics = 5
number_words = 10
# Create and fit the LDA model
lda = LDA(n_components=number_topics, n_jobs=-1)
lda.fit(count_data)
# Print the topics found by the LDA model
print("Topics found via LDA:")
print_topics(lda, count_vectorizer, number_words)

Topics found via LDA:

Topic #0:
drink bottle beverage water soft alcoholic coffee non energy plastic

Topic #1:
food breakfast cereal snack vegetarian cuisine ingredient meal dish games

Topic #2:
food snack cuisine ingredient dish junk chip goods baked potato

Topic #3:
chocolate snack food cookie cookies toy crackers finger oreo baked

Topic #4:
cream dessert ice dairy frozen food ingredient gelato cuisine dish


In [ ]:
topic_guess = lda.transform(count_data)
topic_guess

array([[0.10001554, 0.1000125 , 0.10000597, 0.59994637, 0.10001962],
       [0.02504041, 0.02542299, 0.38864646, 0.5357951 , 0.02509505],
       [0.01429611, 0.01445547, 0.94248229, 0.01442549, 0.01434065],
       ...,
       [0.01257085, 0.01270955, 0.28443367, 0.0131545 , 0.67713142],
       [0.02003069, 0.02030562, 0.9195167 , 0.02010666, 0.02004033],
       [0.10000044, 0.1002217 , 0.10000012, 0.59977733, 0.1000004 ]])

In [ ]:
topic_df = pd.DataFrame(topic_guess)
topic_df


,0,1,2,3,4
0,0.100016,0.100013,0.100006,0.599946,0.100020
1,0.025040,0.025423,0.388646,0.535795,0.025095
2,0.014296,0.014455,0.942482,0.014425,0.014341
3,0.732451,0.066667,0.066867,0.066667,0.067347
4,0.955371,0.011116,0.011117,0.011240,0.011157
...,...,...,...,...,...
745,0.013345,0.182430,0.777267,0.013567,0.013390
746,0.066754,0.068199,0.730251,0.067903,0.066894
747,0.012571,0.012710,0.284434,0.013155,0.677131
748,0.020031,0.020306,0.919517,0.020107,0.020040


In [ ]:
df_lda_topic = pd.concat([df_lda, topic_df], axis=1)
df_lda_topic

,labels,engagement,engagement_level,0,1,2,3,4
0,['Brush'],0.012425,0,0.100016,0.100013,0.100006,0.599946,0.100020
1,"['Food', 'Material property', 'Junk food', 'Fo...",0.030501,0,0.025040,0.025423,0.388646,0.535795,0.025095
2,"['Food', 'Junk food', 'Snack', 'Potato chip', ...",0.032477,0,0.014296,0.014455,0.942482,0.014425,0.014341
3,"['Product', 'Plastic']",0.056535,0,0.732451,0.066667,0.066867,0.066667,0.067347
4,"['Bottle', 'Product', 'Drink', 'Liquid', 'Choc...",0.263126,1,0.955371,0.011116,0.011117,0.011240,0.011157
...,...,...,...,...,...,...,...,...
745,"['Snack', 'Food', 'Junk food', 'Vegetarian foo...",0.201566,1,0.013345,0.182430,0.777267,0.013567,0.013390
746,"['Food', 'Snack']",0.213884,1,0.066754,0.068199,0.730251,0.067903,0.066894
747,"['Frozen dessert', 'Food', 'Ice cream', 'Choco...",0.075475,0,0.012571,0.012710,0.284434,0.013155,0.677131
748,"['Junk food', 'Snack', 'Food', 'Potato chip', ...",0.082336,0,0.020031,0.020306,0.919517,0.020107,0.020040


In [ ]:
low, high = df_lda_topic.engagement.quantile([0.25,.75])

df_low = df_lda_topic.loc[(df_lda_topic.engagement < low)]
df_high = df_lda_topic.loc[(df_lda_topic.engagement > high)]
df_low

,labels,engagement,engagement_level,0,1,2,3,4
0,['Brush'],0.012425,0,0.100016,0.100013,0.100006,0.599946,0.100020
1,"['Food', 'Material property', 'Junk food', 'Fo...",0.030501,0,0.025040,0.025423,0.388646,0.535795,0.025095
2,"['Food', 'Junk food', 'Snack', 'Potato chip', ...",0.032477,0,0.014296,0.014455,0.942482,0.014425,0.014341
3,"['Product', 'Plastic']",0.056535,0,0.732451,0.066667,0.066867,0.066667,0.067347
10,"['Toy', 'Fictional character', 'Games']",0.038046,0,0.040000,0.040940,0.040000,0.839059,0.040000
...,...,...,...,...,...,...,...,...
734,"['Font', 'Laundry detergent', 'Snack']",0.026524,0,0.040027,0.040285,0.040393,0.839259,0.040036
735,"['Frozen dessert', 'Ice cream', 'Food', 'Gelat...",0.043958,0,0.013376,0.185381,0.013747,0.013883,0.773613
736,[],0.067190,0,0.200000,0.200000,0.200000,0.200000,0.200000
741,"['Bottle', 'Drink', 'Non-alcoholic beverage', ...",0.050791,0,0.955533,0.011111,0.011113,0.011115,0.011128


In [ ]:
df_high

,labels,engagement,engagement_level,0,1,2,3,4
4,"['Bottle', 'Product', 'Drink', 'Liquid', 'Choc...",0.263126,1,0.955371,0.011116,0.011117,0.011240,0.011157
6,"['Oreo', 'Snack', 'Cookies and crackers', 'San...",0.363500,1,0.013350,0.013471,0.013613,0.946056,0.013511
9,"['Beverage can', 'Tin can', 'Aluminum can', 'D...",0.185189,1,0.525790,0.016736,0.016713,0.424073,0.016689
15,"['Food', 'Cuisine', 'Ingredient', 'Snack', 'Di...",0.190502,1,0.018200,0.926539,0.018679,0.018288,0.018295
23,"['Snack', 'Chewing gum', 'Food', 'Fruit snack'...",0.260569,1,0.025053,0.194833,0.729475,0.025615,0.025024
...,...,...,...,...,...,...,...,...
714,"['Snack', 'Font', 'Toy']",0.291808,1,0.050042,0.050522,0.050674,0.798707,0.050056
716,"['Doughnut', 'Food', 'Cuisine', 'Bagel', 'Dish...",0.306913,1,0.025009,0.025346,0.899431,0.025112,0.025102
723,"['Breakfast cereal', 'Snack', 'Vegetarian food...",0.333043,1,0.018193,0.925677,0.018483,0.019416,0.018230
745,"['Snack', 'Food', 'Junk food', 'Vegetarian foo...",0.201566,1,0.013345,0.182430,0.777267,0.013567,0.013390


In [ ]:
df_low.describe()

,engagement,engagement_level,0,1,2,3,4
count,188.000000,188.0,188.000000,188.000000,188.000000,188.000000,188.000000
mean,0.049344,0.0,0.158020,0.181360,0.396030,0.135863,0.128727
std,0.012616,0.0,0.267946,0.276906,0.373124,0.236270,0.233114
min,0.012425,0.0,0.010121,0.010527,0.010147,0.010074,0.010541
25%,0.040426,0.0,0.015402,0.016892,0.029007,0.015424,0.015468
50%,0.048586,0.0,0.028621,0.033994,0.245855,0.025156,0.028672
75%,0.060723,0.0,0.103515,0.201651,0.798764,0.103418,0.087557
max,0.068162,0.0,0.957879,0.942491,0.949743,0.942839,0.955178


In [ ]:
df_low_mean = df_low.describe().iloc[1, 2:]
df_low_mean

0    0.158020
1    0.181360
2    0.396030
3    0.135863
4    0.128727
Name: mean, dtype: float64

In [ ]:
df_high.describe()

,engagement,engagement_level,0,1,2,3,4
count,188.000000,188.0,188.000000,188.000000,188.000000,188.000000,188.000000
mean,0.314981,1.0,0.156028,0.234677,0.383011,0.129406,0.096878
std,0.133454,0.0,0.276299,0.338036,0.375044,0.234226,0.215278
min,0.179060,1.0,0.010538,0.010871,0.011111,0.011111,0.010593
25%,0.217697,1.0,0.014334,0.014686,0.018430,0.014388,0.014595
50%,0.274301,1.0,0.025018,0.031586,0.236723,0.020099,0.020076
75%,0.377871,1.0,0.100071,0.419370,0.730376,0.068109,0.050010
max,0.872135,1.0,0.955538,0.952512,0.949714,0.946056,0.955035


In [ ]:
df_high_mean = df_high.describe().iloc[1, 2:]
df_high_mean

0    0.156028
1    0.234677
2    0.383011
3    0.129406
4    0.096878
Name: mean, dtype: float64

In [ ]:
means = pd.DataFrame([df_low_mean, df_high_mean], index = ['low', 'high'])
means

,0,1,2,3,4
low,0.158020,0.181360,0.396030,0.135863,0.128727
high,0.156028,0.234677,0.383011,0.129406,0.096878
